In [62]:
import pandas as pds
import itertools as it
import numpy as np
from copy import copy

In [63]:
def ultrametricite(matrice):
    taille = len(matrice)
    for i in range(taille):
        for j in range(taille):
            for k in range(taille):
                if i !=j !=k:
                    if not matrice[i][k] <= max( matrice[i][j], matrice[j][k] ):
                        return False
    return True

def additive(matrice):
    taille = len(matrice)
    for i in range(taille):
        for j in range(taille):
            for k in range(taille):
                for l in range(taille):
                    if i !=j !=k !=l:
                        maxi = max(matrice[i][k]+matrice[j][l], matrice[i][l]+matrice[j][k])
                        if not matrice[i][j]+matrice[k][l] <= maxi:
                            return False
    return True

def ultrametricite(matrice):
    tools = it.combinations(range(len(matrice)), 3)
    for tool in tools:
        i,j,k = tool
        if not ( matrice[i][k] <= max( matrice[i][j], matrice[j][k] )):
            return False
    return True

def additive(matrice):
    tools = it.combinations(range(len(matrice)), 4)
    for tool in tools:
        i,j,k,l = tool
        maxi = max(matrice[i][k]+matrice[j][l], matrice[i][l]+matrice[j][k])
        if not matrice[i][j]+matrice[k][l] <= maxi:
            return False
    return True

# Toute distance ultramétrique est additive, mais le contraire n'est pas vrai

def somme_distances(matrice):
    """
    somme des distances pour chaque cluster
    """
    taille = len(matrice)
    Sommes = [0 for i in range(taille)]
    for i in range(taille):
        for j in range(taille):
            if i<j:
                Sommes[i] += matrice[i][j]
            elif i>j:
                Sommes[i] += matrice[j][i]
    return Sommes

def somme_clusterI(matrice, i):
    return np.sum(matrice[i])
def somme_clusters(matrice):
    return np.sum(matrice, axis=1)

In [64]:
M1 = [[0,8,7,12],
      [8,0,9,14],
      [7,9,0,11],
      [12,14,11,0]]

M2 = [[0,2,3,8,14,18],
      [2,0,3,8,14,18],
      [3,3,0,8,14,18],
      [8,8,8,0,14,18],
      [14,14,14,14,0,18],
      [18,18,18,18,18,0]]

#UPGMA
M3 = [[0,19,27,8,33,18,13],
      [19,0,31,18,36,1,13],
      [27,31,0,26,41,32,29],
      [8,18,26,0,31,17,14],
      [33,36,41,31,0,35,28],
      [18,1,32,17,35,0,12],
      [13,13,29,14,28,12,0]]
#Neighbor Joining
M4 = [[0,2,4,6,6,8],
      [2,0,4,6,6,8],
      [4,4,0,6,6,8],
      [6,6,6,0,4,8],
      [6,6,6,4,0,8],
      [8,8,8,8,8,0]]

In [65]:
M = [M1,M2,M3,M4]
names = ['M1','M2','M3','M4']
for m in M:
    if ultrametricite(m):
        print(names[M.index(m)], 'est ultramétrique')
    elif additive(m):
        print(names[M.index(m)], 'est additive')
    else:
        print(names[M.index(m)], 'n\'est ni ultramétrique ni addtive')

M1 est additive
M2 est ultramétrique
M3 n'est ni ultramétrique ni addtive
M4 est ultramétrique


In [5]:
print(somme_clusters(np.array(M1)))

[27 31 27 37]


In [100]:
def NJ(matrice):
    """
    Au même titre que UPGMA, NJ renvoie un arbre sous forme de dictionnaire de la forme :
        arbre[parent] = (poids du parent, (enfant1, distance_enfant1), (enfant) 
    """
    L = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    nodes = [L[i] for i in range(len(matrice))]  # on initialise les clusters
    noeuds_internes = 'abcdefghijklmnopqrstuvwxyz'  # on initialise les noeuds intermédiaires
    arbre = {}
    matdist = np.copy(matrice)
    while len(nodes) > 2:
        # Calcul des distances de chaque cluster avec le reste de l'arbre
        Qi = calcul_Qi(matdist)
        
        # On prend i et j tel que Qi soit minimal
        Ni, Nj = np.unravel_index(np.argmin(Qi), Qi.shape)
        
        # On calcule les longueurs des branches entre le nouveau noeud formé et les deux noeuds enfants
        Li, Lj = longueur_des_branches(Ni, Nj, nodes, matdist, arbre)
        
        # On met à jour l'arbre avec les données du noeud parent et des deux noeuds enfants
        parent, wi, wj, noeuds_internes = ajouter_noeud(arbre, Ni, Li, Nj, Lj, nodes, noeuds_internes)
        
        #for key in arbre:
        #    print(key, arbre[key])
        #print('NODES', nodes)
        
        # On supprime les deux noeuds enfant et on rajoute le noeud parent
        nodes.remove(nodes[Ni])
        nodes.remove(nodes[Nj-1])
        nodes.append(parent)
        #print('NODES', nodes)
        
        # On ajoute enlève les deux lignes et colonnes i et j puis l'on rajoute leur noeud parent à la matrice
        matdist = update(matdist, Ni, Nj, wi, wj)
        
    # il ne reste que deux noeuds : Ni=0, Nj=1
    matrice_somme = np.sum(matdist, axis=1)
    Li = (matdist[0,1]/2) + (matdist[0][1]-matdist[1][0])/2
    Lj = (matdist[0,1]/2) + (matdist[1][0]-matdist[0][1])/2
    parent, wi, wj, noeuds_internes = ajouter_noeud(arbre, 0, Li, 1, Lj, nodes, noeuds_internes)
    return arbre, parent




def calcul_Qi(matrice):
    """
    Fonction qui calcule la matrice des Qi de chaque cluster
    """
    shape = matrice.shape
    Qi_res = np.zeros(shape)
    for i in range(shape[0]):
        # on est sensé retrancher matrice[i,i] mais cette valeur est nulle
        ui = np.sum(matrice, axis=1)[i] / (shape[0]-2)  
        for j in range(shape[0]):
            if j != i:
                uj = np.sum(matrice, axis=1)[j] / (shape[0]-2)
                Qi_res[i,j] = matrice[i,j] - ui - uj
    return Qi_res

def longueur_des_branches(i, j, noeuds, matrice, arbre):
    """
    Fonction qui calcule et renvoie les distances de chaque noeud enfant par rapport au noeud parent
    """
    matrice_somme = np.sum(matrice, axis=1)
    di_ij = (matrice[i,j]/2) + (matrice_somme[i]-matrice_somme[j])/2
    dj_ij = (matrice[i,j]/2) + (matrice_somme[j]-matrice_somme[i])/2
    return di_ij, dj_ij

def ajouter_noeud(arbre, Ni, Li, Nj, Lj, noeuds, unused_nodes):
    """
    Fonction qui rajoute le noeud parent rattaché à ses deux noeuds enfants dans l'arbre
    """
    # Si le noeud i est déjà dans l'arbre alors il possède un poids à l'emplacement 0
    if noeuds[Ni] in arbre.keys():
        wi = arbre[noeuds[Ni]][0]
    else:
        wi = 1
    if noeuds[Nj] in arbre.keys():
        wj = arbre[noeuds[Nj]][0]
    else:
        wj = 1
    new_node = unused_nodes[0]
    unused_nodes = unused_nodes[1:]
    arbre[new_node] = (wi+wj, (noeuds[Ni], Li), (noeuds[Nj], Lj))
    return new_node, wi, wj, unused_nodes

def update(matrice, Ni, Nj, Wi, Wj):
    """
    Fonciton qui met à jour la matrice en enlevant les deux noeuds enfants et en rajoutant le noeud parent
    puis calcule les nouvelles valeurs de distance de ce noeud parent par rapport à chaque autre noeud restant
    """
    copie = np.copy(matrice)
    
    # On garde en mémoire les valeurs supprimées pour pouvoir recalculer les nouvelles distances du nouveau noeud
    I_values = list(copy(copie[Ni]))
    J_values = list(copy(copie[Nj]))
    I_values.pop(Ni)
    I_values.pop(Nj-1)
    J_values.pop(Ni)
    J_values.pop(Nj-1)
    
    copie = np.delete(copie, Ni, 0) #On enlève la ligne i
    copie = np.delete(copie, Ni, 1) #On enlève la colonne i
    copie = np.delete(copie, Nj-1, 0) #On enlève la ligne j d'indice j-1 car on a déjà enlevé une ligne
    copie = np.delete(copie, Nj-1, 1) #On enlève la colonne j d'indice j-1 car on a déjà enlevé une colonne
    copie = np.concatenate((copie, np.zeros((1,copie.shape[0]))), axis=0) #On rajoute une ligne à la fin (en bas)
    copie = np.concatenate((copie, np.zeros((copie.shape[1]+1,1))), axis=1) #On rajoute une colonne à la fin (à droite)
    for k in range(copie.shape[0]-1):
        copie[-1,k] = (I_values[k]+J_values[k]-matrice[Ni,Nj])/2 # remplissage de la dernière ligne
        copie[k,-1] = (I_values[k]+J_values[k]-matrice[Ni,Nj])/2 # remplissage de la dernière colonne
    return copie
    
M3 = [[0,19,27,8,33,18,13],
      [19,0,31,18,36,1,13],
      [27,31,0,26,41,32,29],
      [8,18,26,0,31,17,14],
      [33,36,41,31,0,35,28],
      [18,1,32,17,35,0,12],
      [13,13,29,14,28,12,0]]   
M4 = [[0,2,4,6,6,8],
      [2,0,4,6,6,8],
      [4,4,0,6,6,8],
      [6,6,6,0,4,8],
      [6,6,6,4,0,8],
      [8,8,8,8,8,0]]
arbre, parent = NJ(np.array(M4))

#print('FINAL',NJ(np.array(M4)))
#data2 = NJ(np.array(M3))
#print('FINAL',NJ(np.array(M3)))






In [103]:
def pile_str_modifie(arbre, parent, leaf):
    """
    Fonction qui convertit l'arbre de l'algorithme NJ
    """
    Res = ''
    pile = [ parent ]
    while pile != []:
        ref = pile.pop(-1)
        if ref in arbre.keys():
            pile.append(')')
            pile.append(':' + str(arbre[ref][2][1]))
            pile.append(arbre[ref][2][0])
            pile.append(',')
            pile.append(':' + str(arbre[ref][1][1]))
            pile.append(arbre[ref][1][0])
            pile.append('(')
        else:
            Res += ref
    return Res

feuilles = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
res_pile_M4 = pile_str_modifie(arbre, parent, feuilles)
print('M4', res_pile_M3)

M4 ((D:2.0,E:2.0):0.5,(F:5.0,(C:3.0,(A:1.0,B:1.0):0.0):1.0):0.5)


In [7]:
a = np.array([[1, 2], [3, 4]])
print(a,a.shape)
b = np.array([[5, 6]])
print(b,b.shape)
print(type(a))
np.concatenate((a, b), axis=0)

[[1 2]
 [3 4]] (2, 2)
[[5 6]] (1, 2)
<class 'numpy.ndarray'>


array([[1, 2],
       [3, 4],
       [5, 6]])

In [8]:
copie = np.array([[0, 6, 6, 8],
 [6, 0, 4, 8],
 [6, 4, 0, 8],
 [8, 8, 8, 0]])
print(np.zeros((1,4)))
print(np.zeros((4,1)))

print(copie, copie.shape)
print(np.concatenate((copie,np.zeros(4)),axis=0))

[[0. 0. 0. 0.]]
[[0.]
 [0.]
 [0.]
 [0.]]
[[0 6 6 8]
 [6 0 4 8]
 [6 4 0 8]
 [8 8 8 0]] (4, 4)


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [27]:
A = np.array([[1,2,3], [4,5,6]])
print(A.shape)
print(np.sum(A,axis=1))

(2, 3)
[ 6 15]


In [ ]:
x = np.array([[0, 6, 6, 8],
 [6, 0, 4, 8],
 [6, 4, 0, 8],
 [8, 8, 8, 0]])
x += np.eye(4)*10
print(x)
res = np.unravel_index(np.argmin(x, axis=None), x.shape)

print(res)

In [ ]:
A = np.zeros(5)
B = np.zeros(5).T
print(A.shape, B.shape)